In [1]:
cd ~/code/biom262-2016/weeks/week01/data

/home/ucsd-train17/code/biom262-2016/weeks/week01/data


In [2]:
%%bash --out exercise1
cat tf.bed | grep --word-regexp 'NFKB' | wc -l
cat tf.bed | grep --word-regexp 'NFKB' > tf.nfkb.bed
wc -l tf.nfkb.bed
echo '---First 10 lines ---'
head tf.nfkb.bed
echo '---Random 10 lines ---'
awk -v seed=907 'BEGIN{srand(seed);}{ if (rand() <0.5){print $0}}' tf.nfkb.bed | head
echo '--- Last 10 lines ---'
tail tf.nfkb.bed

In [3]:
print(exercise2)

NameError: name 'exercise2' is not defined

In [4]:
print(exercise1)

702
702 tf.nfkb.bed
---First 10 lines ---
chr22	17565763	17566145	NFKB	406	.	17565763	17566145	0	1	382,	0,
chr22	17568509	17568945	NFKB	354	.	17568509	17568945	0	1	436,	0,
chr22	17700141	17700502	NFKB	525	.	17700141	17700502	0	1	361,	0,
chr22	17701962	17702278	NFKB	295	.	17701962	17702278	0	1	316,	0,
chr22	17709521	17709797	NFKB	300	.	17709521	17709797	0	1	276,	0,
chr22	17724391	17724724	NFKB	413	.	17724391	17724724	0	1	333,	0,
chr22	17738401	17738677	NFKB	462	.	17738401	17738677	0	1	276,	0,
chr22	17738857	17739282	NFKB	573	.	17738857	17739282	0	1	425,	0,
chr22	17761088	17761435	NFKB	244	.	17761088	17761435	0	1	347,	0,
chr22	17784788	17785118	NFKB	1000	.	17784788	17785118	0	1	330,	0,
---Random 10 lines ---
chr22	17700141	17700502	NFKB	525	.	17700141	17700502	0	1	361,	0,
chr22	17701962	17702278	NFKB	295	.	17701962	17702278	0	1	316,	0,
chr22	17709521	17709797	NFKB	300	.	17709521	17709797	0	1	276,	0,
chr22	17738401	17738677	NFKB	462	.	17738401	17738677	0	1	276,	0,
chr22	17738857	17739282	

In [5]:
%%bash --out exercise2
cat gencode.v19.annotation.chr22.gtf | grep --word-regexp 'transcript' | wc -l
cat gencode.v19.annotation.chr22.gtf | grep --word-regexp 'transcript > gencode.v19.annotation.chr22.transcript.gtf'
wc -l gencode.v19.annotation.chr22.transcript.gtf
echo '--- First 10 lines ---'
head gencode.v19.annotation.chr22.transcript.gtf
echo '--- Random 10 lines ---'
awk -v seed=907 'BEGIN{srand(seed);}{ if (rand() < 0.5 ) {print $0}}'  gencode.v19.annotation.chr22.transcript.gtf | head
echo '--- Last 10 lines ---'
tail gencode.v19.annotation.chr22.transcript.gtf

In [6]:
print(exercise2)

4459
4459 gencode.v19.annotation.chr22.transcript.gtf
--- First 10 lines ---
chr22	HAVANA	transcript	16062157	16063236	.	+	.	gene_id "ENSG00000233866.1"; transcript_id "ENST00000424770.1"; gene_type "lincRNA"; gene_status "KNOWN"; gene_name "LA16c-4G1.3"; transcript_type "lincRNA"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.3-001"; level 2; tag "basic"; havana_gene "OTTHUMG00000140195.1"; havana_transcript "OTTHUMT00000276574.1";
chr22	HAVANA	transcript	16076052	16076172	.	-	.	gene_id "ENSG00000229286.1"; transcript_id "ENST00000448070.1"; gene_type "pseudogene"; gene_status "KNOWN"; gene_name "LA16c-4G1.4"; transcript_type "unprocessed_pseudogene"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.4-001"; level 2; ont "PGO:0000005"; havana_gene "OTTHUMG00000140193.1"; havana_transcript "OTTHUMT00000276571.1";
chr22	HAVANA	transcript	16084249	16084826	.	-	.	gene_id "ENSG00000235265.1"; transcript_id "ENST00000413156.1"; gene_type "pseudogene"; gene_status "KNOWN"; gene_n

In [17]:
%%bash --out exercise3
bedtools flank -i gencode.v19.annotation.chr22.transcript.gtf -g hg19.genome -l 0 -r 2000 > gencode.v19.annotation.chr22.transcript.promoter.gtf

wc -l gencode.v19.annotation.chr22.transcript.promoter.gtf
echo '--- First 10 lines ---'
head gencode.v19.annotation.chr22.transcript.promoter.gtf
echo '--- Random 10 lines ---'
awk -v seed=907 'BEGIN{srand(seed);}{ if (rand() < 0.5 ) {print $0}}' gencode.v19.annotation.chr22.transcript.promoter.gtf | head
echo '--- Last 10 lines ---'
tail gencode.v19.annotation.chr22.transcript.promoter.gtf

bash: line 1: bedtools: command not found


In [21]:
%%bash --out exercise4
bedtools intersect -a gencode.v19.annotation.chr22.transcript.promoter.gtf -b tf.nfkb.bed > gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf

wc -l gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf
echo '--- First 10 lines ---'
head gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf
echo '--- Random 10 lines ---'
awk -v seed=908 'BEGIN{srand(seed);}{ if (rand() < 0.5 ) {print $0}}' gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf | head
echo '--- Last 10 lines ---'
tail gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf

bash: line 1: bedtools: command not found


In [20]:
%%bash --out exercise5
bedtools getfasta -fi GRCh37.p13.chr22.fa -bed gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf -s -fo gencode.v19.annotation.chr22.transcript.promoter.nfkb.fasta

wc -l gencode.v19.annotation.chr22.transcript.promoter.nfkb.fasta
echo '--- First 10 lines ---'
head gencode.v19.annotation.chr22.transcript.promoter.nfkb.fasta
echo '--- Random 10 lines ---'
awk -v seed=908 'BEGIN{srand(seed);}{ if (rand() < 0.5 ) {print $0}}' gencode.v19.annotation.chr22.transcript.promoter.nfkb.fasta | head
echo '--- Last 10 lines ---'
tail gencode.v19.annotation.chr22.transcript.promoter.nfkb.fasta

bash: line 1: bedtools: command not found
